In [ ]:
import os
import pandas as pd
import numpy as np
import anndata
import time
import matplotlib.pyplot as plt
import json
import requests
import pickle

## Splited Avg Exp Mat

In [ ]:
fillist = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/abc_download_root/expression_matrices/10X-SC-FileList.txt"

In [ ]:
flist = [l.strip() for l in open(fillist, 'rt').readlines()]

In [ ]:
flist[:5]

In [ ]:
adata = anndata.read_h5ad(test_fil,backed='r')
print(adata)

In [ ]:
version = '20230830'
download_base = '/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/abc_download_root'

use_local_cache = False
manifest_path = 'releases/%s/manifest.json' % version

if not use_local_cache :
    url = 'https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/' + manifest_path
    manifest = json.loads(requests.get(url).text)
else :
    file = os.path.join(download_base,manifest_path)
    with open(file,'rb') as f:
        manifest = json.load(f)
    
metadata = manifest['file_listing']['WMB-10X']['metadata']

In [ ]:
version = '20230830'
download_base = '/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/abc_download_root'
view_directory = os.path.join( download_base, 
                               manifest['directory_listing']['WMB-10X']['directories']['metadata']['relative_path'], 
                              'views')
cell_extended = pd.read_csv(os.path.join( view_directory, 'cell_metadata_with_cluster_annotation.csv'))

In [ ]:
#cell_extended.to_csv("dat/cell_metadata_with_cluster_annotation.csv")
cell_extended = pd.read_csv("dat/cell_metadata_with_cluster_annotation.csv")

In [ ]:
# class
# subclass
# supertype
# cluster

In [ ]:
# Aggregate Cells to each Cell Type

In [ ]:
# Class
label="class"
groupnames = cell_extended[label].unique()
group2barcode = {}
for groupname in groupnames:
    tmpdf = cell_extended[cell_extended[label]==groupname]
    group2barcode[groupname] = tmpdf["cell_barcode"].values

In [ ]:
adata.var.index.values

In [ ]:
dat = []
group_names = []
for group, cells in group2barcode.items():
    print(group, len(cells))
    group_names.append(group)
    cells = group2barcode[group]
    submat = adata[adata.obs["cell_barcode"].isin(cells)]
    test = submat.to_df()
    mean = test.mean(axis=0)
    dat.append(mean.values)

In [ ]:
dat = np.array(dat).transpose()
dat.shape

In [ ]:
df = pd.DataFrame(data=dat, index=adata.var.index.values, columns=list(group2barcode.keys()))

In [ ]:
def SaveDict(Dict, fname):
    with open(fname, 'wb') as hand:
        pickle.dump(Dict, hand)
    return

def LoadDict(fname):
    with open(fname, 'rb') as hand:
        b = pickle.load(hand)
        return b

def AggregateExpMat2Cluster(adata, label, name):
    # Class
    groupnames = cell_extended[label].unique()
    print(len(groupnames),"uniq groups")
    group2barcode = {}
    for groupname in groupnames:
        tmpdf = cell_extended[cell_extended[label]==groupname]
        group2barcode[groupname] = tmpdf["cell_barcode"].values
        
    dat = []
    group_names = []
    CellType2NCell = {}
    for group, cells in group2barcode.items():
        print(group, len(cells))
        cells = group2barcode[group]
        submat = adata[adata.obs["cell_barcode"].isin(cells)]
        if submat.shape[0] == 0: # No cell barcode in this cell type
            continue   
        CellType2NCell[group] = submat.shape[0]
        test = submat.to_df()
        mean = test.mean(axis=0)
        group_names.append(group)
        dat.append(mean.values)
    dat = np.array(dat).transpose()
    df = pd.DataFrame(data=dat, index=adata.var.index.values, columns=group_names)
    df.to_csv("dat/AggregatedExpMat/ExpMat.{}.{}.csv".format(name, label))
    SaveDict(CellType2NCell, "dat/AggregatedExpMat/NumCells.{}.{}.csv".format(name, label))
    return

In [ ]:
%%time
AggregateExpMat2Cluster(adata, "subclass", "WMB-10Xv3-STR")

In [ ]:
label = "subclass"
groupnames = cell_extended[label].unique()
print(len(groupnames),"uniq groups")
group2barcode = {}
for groupname in groupnames:
    tmpdf = cell_extended[cell_extended[label]==groupname]
    group2barcode[groupname] = tmpdf["cell_barcode"].values

In [ ]:
group = "282 POR Spp1 Gly-Gaba"
group = "244 MV-SPIV Slc6a2 Glut"
cells = group2barcode[group]
submat = adata[adata.obs["cell_barcode"].isin(cells)]

In [ ]:
submat

In [ ]:
submat.shape

## Combine splited Exp Avg mat


In [ ]:
Split_Exp_Dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/AggregatedExpMat/"

In [ ]:
testfil = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/abc_download_root/expression_matrices/WMB-10Xv2/20230630/WMB-10Xv2-Isocortex-2-log2.h5ad"

In [ ]:
adata = anndata.read_h5ad(testfil, backed='r')

In [ ]:
test_cell_meta = pd.read_csv("dat/Cell_Meta_Splited/WMB-10Xv2-Isocortex-2.meta.csv", index_col=0)

In [ ]:
adata = anndata.read_h5ad(testfil, backed='r')
print(adata)

In [ ]:
test_cell_meta.head(3)

In [ ]:
test_celltype = "046 Vip Gaba"
test_celltype = "053 Sst Gaba"
test_celltype = "002 IT EP-CLA Glut"
test_celltype = "005 L5 IT CTX Glut"
cells = test_cell_meta[test_cell_meta["subclass"]==test_celltype]["cell_barcode"].values

In [ ]:
submat = adata[adata.obs["cell_barcode"].isin(cells)]

In [ ]:
submat

In [ ]:
test = submat.to_df()

In [ ]:
count = 0
for i, row in test.iterrows():
    plt.scatter(np.arange(len(row.values)), row.values)
    plt.show()
    count += 1
    if count >10:
        break

In [ ]:
testfil = "/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/abc_download_root/expression_matrices/WMB-10Xv3/20230630/WMB-10Xv3-Isocortex-2-log2.h5ad"

In [ ]:
adata = anndata.read_h5ad(testfil, backed='r')

In [ ]:
test_cell_meta = pd.read_csv("dat/Cell_Meta_Splited/WMB-10Xv3-Isocortex-2.meta.csv", index_col=0)

In [ ]:
test_celltype = "046 Vip Gaba"
test_celltype = "053 Sst Gaba"
test_celltype = "002 IT EP-CLA Glut"
test_celltype = "005 L5 IT CTX Glut"
cells = test_cell_meta[test_cell_meta["subclass"]==test_celltype]["cell_barcode"].values

In [ ]:
submat = adata[adata.obs["cell_barcode"].isin(cells)]
test = submat.to_df()

In [ ]:
count = 0
for i, row in test.iterrows():
    plt.scatter(np.arange(len(row.values)), row.values)
    plt.show()
    count += 1
    if count >10:
        break